In [1]:
%run runaway_functionsv2
%matplotlib qt


In [3]:
#Get Runaways
cluster = Cluster('Berkeley_97')
cluster.generate_tables()
theoretical_data = theoretical_isochrone(cluster,output="table",printing=False)
fs = cluster.read_table('fs')
runaways_all = get_runaways(cluster,fs,theoretical_data)
display(runaways_all)
mask = [T > config['runaway_temp'] for T in runaways_all['Temp. Est']]
runaways = runaways_all[mask]
runaways

All tables present (stars_in_region,stars_fromDR3,stars_fromDR3_dis,fs,nsfs)
Berkeley_97/Berkeley_97_runaways.tsv exists. Skipping traceback and temp estimates.
261 stars had been checked, 13 traced back to cluster


Temp. Est,v_pec,µ_pec,_q_1,_r_1,RA_ICRS_1,rmRA,e_rmRA,rmDE,e_rmDE,DE_ICRS_1,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,_q_2,_r_2,RA_ICRS_2,DE_ICRS_2,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
K,km / s,mas / yr,,,deg,mas / yr,mas / yr,mas / yr,mas / yr,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,,deg,deg,pc,pc,pc,pc,pc,pc,
float64,float64,float64,int32,float64,float64,float64,float32,float64,float32,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str1,str15,str19,int32,str10,str1,str16,str1,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32
14941.699574067023,29.861,2.404,1,1.5205,339.89581711423,2.361,0.188,0.453,0.185,58.99547917199,2008384504995669248,0.0202,0.0197,0.3560,0.0241,1.404,-0.379,0.025,-1.352,0.024,1.086,13221.76503,3.482,15.384143,6055,14.81,15.883185,1.044e+04,18.25,14.701116,1.182070,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,9168.0,3.7061,-0.6147,4094.0940,2.4713,--,178793398958385263,--,--,--,URAT1-745392846,J223935.00+585943.5,13469566,N1C5026796,--,22393500+5859436,--,339.89582038348,58.99548518133,1,1.5205,339.89581711423,58.99547917199,2620.36230000,2461.53345000,2759.61401000,2605.56982000,2475.57690000,2754.27393000,10033
11849.49930976177,30.066,3.053,1,6.5222,339.69805696343,-2.180,0.188,-2.137,0.184,58.92918778013,2008382546490573824,0.0190,0.0193,0.4561,0.0228,6.305,-4.920,0.025,-3.942,0.023,1.046,13964.47166,3.916,15.324805,6086,13.27,15.877747,1.147e+04,11.21,14.599200,1.278546,--,--,--,--,0,0,0,1,0,0,0,0,0,1,0,--,--,--,--,--,--,178713396980935748,--,--,--,URAT1-745392015,--,--,--,--,22384755+5855451,--,339.69809933678,58.92920530089,1,6.5222,339.69805696343,58.92918778013,2077.95093000,1973.40735000,2173.33203000,2060.58984000,1965.45789000,2167.60474000,10033
9894.640051300763,22.010,1.593,1,1.6227,339.82327510265,-1.024,0.208,-1.221,0.204,59.02791551662,2008430306528315136,0.0373,0.0353,0.3154,0.0444,4.830,-3.764,0.045,-3.026,0.043,1.051,4448.12149,2.398,16.566925,1838,8.274,17.177988,3774,7.525,15.805863,1.372125,--,--,--,--,0,0,0,1,0,0,0,0,0,1,0,--,--,--,--,--,--,178833398233174213,--,--,--,URAT1-746391405,--,--,N1C5028985,--,22391760+5901404,--,339.82330760932,59.02792896656,1,1.6227,339.82327510265,59.02791551662,2914.14893000,2617.74390000,3300.76758000,2788.21753000,2530.42310000,3075.98462000,10033
9005.33762750648,27.445,2.591,1,6.1584,339.68152272449,-1.939,0.186,-1.719,0.182,58.94980063744,2008429550614076032,0.0176,0.0172,0.4199,0.0206,5.858,-4.679,0.023,-3.524,0.021,0.968,16402.18114,4.41,15.150113,6712,12.49,15.771441,1.408e+04,18.72,14.376559,1.394881,--,--,--,--,0,0,0,1,0,0,0,0,0,1,0,--,--,--,--,--,--,178743396815500484,--,--,--,URAT1-745391944,J223843.56+585659.4,--,N1C5024111,--,22384357+5856594,--,339.68156304533,58.94981629825,1,6.1584,339.68152272449,58.94980063744,2234.62158000,2138.99561000,2340.34644000,2194.90039000,2092.13135000,2296.43628000,10033
8334.892445316002,35.010,2.806,1,3.2286,339.76625014965,-2.676,0.197,-0.845,0.193,58.97239278067,2008383126302250624,0.0263,0.0254,0.3567,0.0308,6.030,-5.416,0.034,-2.650,0.032,1.003,7686.25360,2.913,15.973080,2848,7.975,16.702312,7088,13.62,15.121582,1.580730,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,8724.0,4.1062,0.4906,2514.2654,3.3183,--,178763397662907586,--,--,--,URAT1-745392287,--,--,N1C5025440,--,22390

Temp. Est,v_pec,µ_pec,_q_1,_r_1,RA_ICRS_1,rmRA,e_rmRA,rmDE,e_rmDE,DE_ICRS_1,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,_q_2,_r_2,RA_ICRS_2,DE_ICRS_2,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
K,km / s,mas / yr,,,deg,mas / yr,mas / yr,mas / yr,mas / yr,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,,deg,deg,pc,pc,pc,pc,pc,pc,
float64,float64,float64,int32,float64,float64,float64,float32,float64,float32,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str1,str15,str19,int32,str10,str1,str16,str1,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32
14941.699574067023,29.861,2.404,1,1.5205,339.89581711423,2.361,0.188,0.453,0.185,58.99547917199,2008384504995669248,0.0202,0.0197,0.3560,0.0241,1.404,-0.379,0.025,-1.352,0.024,1.086,13221.76503,3.482,15.384143,6055,14.81,15.883185,1.044e+04,18.25,14.701116,1.182070,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,9168.0,3.7061,-0.6147,4094.0940,2.4713,--,178793398958385263,--,--,--,URAT1-745392846,J223935.00+585943.5,13469566,N1C5026796,--,22393500+5859436,--,339.89582038348,58.99548518133,1,1.5205,339.89581711423,58.99547917199,2620.36230000,2461.53345000,2759.61401000,2605.56982000,2475.57690000,2754.27393000,10033
11849.49930976177,30.066,3.053,1,6.5222,339.69805696343,-2.180,0.188,-2.137,0.184,58.92918778013,2008382546490573824,0.0190,0.0193,0.4561,0.0228,6.305,-4.920,0.025,-3.942,0.023,1.046,13964.47166,3.916,15.324805,6086,13.27,15.877747,1.147e+04,11.21,14.599200,1.278546,--,--,--,--,0,0,0,1,0,0,0,0,0,1,0,--,--,--,--,--,--,178713396980935748,--,--,--,URAT1-745392015,--,--,--,--,22384755+5855451,--,339.69809933678,58.92920530089,1,6.5222,339.69805696343,58.92918778013,2077.95093000,1973.40735000,2173.33203000,2060.58984000,1965.45789000,2167.60474000,10033


In [4]:

# Plot CMD
BP_RP_theo, Gmag_theo = theoretical_isochrone(cluster)

# fig = plt.figure(figsize=(19, 17))
# # fig = plt.figure()
# gs = GridSpec(1, 1, width_ratios=[1, 1])

# # Line Theorietical curve
# # ax1 = fig.add_subplot(gs[0, 0])
# ax1 = fig.subplots()

fig,ax1 = plt.subplots()

# x_min, x_max = ax1.get_xlim()  # Get the current x-limits
# x_max = max(theoretical_data['BP-RP'])
# ax1.set_xlim(x_min,x_max)

# Calculate the 20% lower y-limit
# y_min, y_max = ax1.get_ylim()  # Get the current y-limits
# y_max = max(Gmag_theo)
# y_range = y_max - y_min
# new_y_min = y_min - 0.2 * y_range
# ax1.set_ylim(new_y_min, y_max)



ax1.plot(BP_RP_theo, Gmag_theo, label='Theoretical Isochrone')
ax1.set_xlabel(r"$G_{BP}-G_{RP}$ (mag)", fontsize=14)
ax1.set_ylabel(r"$G$ (mag)", fontsize=14)
ax1.set_title(f"CMD for {cluster.name}", fontsize=14)
ax1.invert_yaxis()


#Scatter stars in the region
sir = cluster.stars_in_region()
sir_gmag,sir_bp_rp = sir['Gmag'],sir['BP-RP']
ax1.scatter(sir_bp_rp,sir_gmag,s=2, color='grey',label=f'{len(sir)} stars in the region {cluster.calculate_search_arcmin()}')


# Scatter dias members
dias_members = cluster.dias_members(memb_prob=config["memb_prob"])
dias_gmag,dias_bp_rp = dias_members['Gmag'],dias_members['BP-RP']
ax1.scatter(dias_bp_rp,dias_gmag,s=8, color='black',label=f'{len(dias_members)} Dias Members P > {config["memb_prob"]}')






# Scatter my members
my_members = find_cluster(sir,refCluster=cluster.name)
my_gmag,my_bp_rp = my_members['Gmag'],my_members['BP-RP']
ax1.scatter(my_bp_rp,my_gmag,s=10,alpha =0.7,marker='x', color='blue',label=rf'{len(my_members)} My Members $\sigma$ < {my_members["v_pec"].max():.2f}')




# Scatter runaways
runaways_all = cluster.read_table('runaways')
_runaways_gmag,_runaways_bp_rp = runaways_all['Gmag'],runaways_all['BP-RP']
mask = [T > config['runaway_temp'] for T in runaways_all['Temp. Est']]
runaways = runaways_all[mask]
runaways_gmag,runaways_bp_rp = runaways['Gmag'],runaways['BP-RP']
## all runaways
ax1.scatter(_runaways_bp_rp,_runaways_gmag,s=5,alpha=0.5, c=runaways_all['Temp. Est'],cmap='spring_r',norm=plt.Normalize(4000, 23000),label=rf'{len(runaways_all)} Runaways ($v$ > 17.6)')




## main runaways T > 10,000K
annotations = []

def onpick3(event):
    global annotations
    for ann in annotations:
        ann.remove()
    annotations = []
    
    ind = event.ind
    for index in ind:
        temp_est = runaways['Temp. Est'][index]
        ann = ax1.annotate(f'{temp_est:.0f} K', (runaways_bp_rp[index], runaways_gmag[index]), textcoords='offset points', fontsize=10, color='black', ha='center')
        ann.set_path_effects([pe.withStroke(linewidth=3, foreground='white')])
        annotations.append(ann)
    plt.draw()


scatter_main = ax1.scatter(runaways_bp_rp,runaways_gmag,picker=True,s=20, c=runaways['Temp. Est'],cmap='spring_r',norm=plt.Normalize(4000, 23000),label=rf'{len(runaways)} Runaways ($v$ > 17.6) & T > {config["runaway_temp"]:,} K')
# Connect the pick_event to the onpick3 function
fig.canvas.mpl_connect('pick_event', onpick3)


colorbar = fig.colorbar(scatter_main,ax=ax1)
colorbar.set_label('Temperature (K)')
ax1.legend(loc='upper right')


Berkeley_97
Av: 3.1
logage: 7.0
[Fe/H]: -0.1
Theoretical isochrone exists, reading it.
Berkeley_97/Berkeley_97_stars_in_region.tsv exists in Berkeley_97 with 446 stars
mean(v_pec):  2.226454335203929
std(v_pec):  1.531512700823818
v_pec_max:  6.6702839090600845


In [ ]:
# import matplotlib.gridspec as gridspec
# def runaway_plot():
#     fig = plt.figure(figsize=(12, 12))
#     gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1])

#     # First subplot
#     ax1 = fig.add_subplot(gs[0, 0])
#     ax1.plot(BP_RP_theo, Gmag_theo, label='Theoretical Isochrone')
#     ax1.set_xlabel(r"$G_{BP}-G_{RP}$ (mag)", fontsize=14)
#     ax1.set_ylabel(r"$G$ (mag)", fontsize=14)
#     ax1.set_title(f"CMD for {cluster.name}", fontsize=14)
#     ax1.invert_yaxis()
#     ax1.legend()

#     # Second subplot
#     ax2 = fig.add_subplot(gs[0, 1])
#     ax2.plot(BP_RP_theo, Gmag_theo, label='Theoretical Isochrone')
#     ax2.set_xlabel(r"$G_{BP}-G_{RP}$ (mag)", fontsize=14)
#     ax2.set_ylabel(r"$G$ (mag)", fontsize=14)
#     ax2.set_title(f"CMD for {cluster.name}", fontsize=14)
#     ax2.legend()

#     # Third subplot
#     ax3 = fig.add_subplot(gs[1, 0])
#     ax3.plot(BP_RP_theo, Gmag_theo, label='Theoretical Isochrone')
#     ax3.set_xlabel(r"$G_{BP}-G_{RP}$ (mag)", fontsize=14)
#     ax3.set_ylabel(r"$G$ (mag)", fontsize=14)
#     ax3.set_title(f"CMD for {cluster.name}", fontsize=14)
#     ax3.invert_xaxis()
#     ax3.legend()


#     # Fourth subplot
#     ax4 = fig.add_subplot(gs[1, 1])
#     ax4.plot(BP_RP_theo, Gmag_theo, label='Theoretical Isochrone')
#     ax4.set_xlabel(r"$G_{BP}-G_{RP}$ (mag)", fontsize=14)
#     ax4.set_ylabel(r"$G$ (mag)", fontsize=14)
#     ax4.set_title(f"CMD for {cluster.name}", fontsize=14)
#     ax4.invert_xaxis()
#     ax4.invert_yaxis()
#     ax4.legend()

#     # plt.tight_layout()
#     plt.show()
# runaway_plot()